I will be approaching this project 2 ways. 
1. Classic text classification, with 1 model predicting 20 newsgroups.
2. Tiered text classification, with 1 model predicting 6 topics and then 6 models, each predicting the newsgroups associated with each topic. 

Below, is the classic text classification.


# Imports

In [13]:
import numpy as np
import pandas as pd
import sys
libraries = (('Numpy', np), ('Pandas', pd))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))

Python Version: 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0] 

Numpy Version: 1.16.4
Pandas Version: 0.23.0


In [14]:
import TxtFiles as tf
import FTCommands as ftc
import EvaluatePredictions as ep

sys.path.append('../MyModules/')
import KleptoFunctions as kf

# Data
Import data from 1_Process_Data.ipynb.  
Texts have been cleaned and processed. 

In [15]:
data_path = 'data/'

In [16]:
data = kf.puking_file('2019.06.25_cleaned_newsgroups', data_path)

Filename: 2019.06.25_cleaned_newsgroups 
# of Folders: 5 
Type: <class 'pandas.core.frame.DataFrame'> 
Len: 18752


# FastText Txt files
The entire list of doc IDs will be split 3 ways - Training set, Validation Set, & Holdout Set.

## split ids

In [17]:
random_state = 19

In [18]:
set_ids = tf.training_validation_holdout_split(data['_id'], random_state)

holdout	 3751
training	 11250
validation	 3751


## Txt Files
Each message is labeled with 1 of 20 newsgroup labels as we are not take the step to predict any of the topics on the Tier1 for the classic model. 

In [19]:
for name_part in set_ids.keys():
    filepath = data_path+'%s.txt' % name_part
    df = data[data['_id'].isin(set_ids[name_part])]
    tf.make_txtfile(filepath, df, 'tier2_targets')
    print(filepath)

data/holdout_19.txt
data/T1_training_19.txt
data/T1_validation_19.txt


In [20]:
train_filename = [k for k in set_ids.keys() if 'training' in k][0]+'.txt'
train_address = data_path+train_filename
test_filename = [k for k in set_ids.keys() if 'validation' in k][0]+'.txt'
test_address = data_path+test_filename
holdout_filename = [k for k in set_ids.keys() if 'holdout' in k][0]+'.txt'
holdout_address = data_path+holdout_filename

# Training Model
   * ngram - max length of word ngram.    
   * lr - step size to convergance.   
   * dim - size of word vectors.   
   * ws - size of the context window.  
   * epoch - number of passes over the training data.  
   * loss - loss function.  
       * ns - negative sampling  
       * hs - hierarchical softmax  
       * softmax  

In [21]:
ngram = 3
lr = 0.25
dim = 200
ws = 5
epoch = 25
loss = 'ns'

In [22]:
model_address, predict_address = ftc.train_test_predict(train_address, test_address, ngram, lr, dim, ws, epoch, loss)
print(model_address)
print(predict_address)

N	3751
P@1	0.804
R@1	0.804

data/T1_model
data/T1_prediction_19.txt


# Using Model
Please note: FastText has its own evaluation and it is flawed. It is not precision, recall, accuracy, or f1 score. But it is a reliable measure to determine if a model is improving during a bunch of experiments. However, evaluation with SkLearn is recommended.   

In [23]:
predict_address = ftc.test_predict(holdout_address, model_address)
print(predict_address)

N	3751
P@1	0.815
R@1	0.815

data/holdout_prediction_19.txt


In [24]:
# how did it do on our holdout set?
precision, recall, fscore, support = ep.score_txtfiles(holdout_address, predict_address)

precision: 0.8080892948582475
recall: 0.805236375031088
fscore: 0.8034230052440412
